In [7]:
import requests
from bs4 import BeautifulSoup
import re
import time
import gen_css
import add_current
import get_mp_tools

In [8]:
journal_name = 'AER'
journal_main_page_url = 'https://www.aeaweb.org/journals/aer'

In [9]:
journal_main_page_data = get_mp_tools.get(journal_main_page_url)

In [10]:
current_issue_url = journal_main_page_data.find(class_="current-issue-link")
current_issue_url = 'https://www.aeaweb.org' + current_issue_url.get('href')
current_issue_url

'https://www.aeaweb.org/issues/764'

In [11]:
issue = re.search('iss.+',current_issue_url)[0]
issue = re.sub(r"/", "_", issue)
issue

'issues_764'

In [12]:
current_issue_data = get_mp_tools.get(current_issue_url)

In [13]:
article_htmls = current_issue_data.find_all(class_='journal-article')

In [14]:
titles = []
names = []
abstracts = []

for article_link in article_htmls:
    try:
        article_link = 'https://www.aeaweb.org' + article_link.find('a').get('href')
        article_data = get_mp_tools.get(article_link)
        
        titles.append(article_data.find(class_='title').text)

        name = article_data.find(class_='attribution').text
        name = re.findall(r'\b[A-Za-z]+(?:\s[A-Za-z]+)+\b', name)
        name = ', '.join(name)
        names.append(name)
        
        abstract = article_data.find(class_='abstract').text
        abstract = cleaned_text = re.sub(r"Abstract|\t|\n", "", abstract)
        abstracts.append(abstract)
    except:
        pass
    
article = [*zip(titles, names, abstracts)]
article

[('Front Matter',
  '',
  'Why have white, less-educated voters left the Democratic Party? We highlight the role of the 1994 North American Free Trade Agreement (NAFTA). In event-study analysis, we demonstrate that counties whose 1990 employment depended on industries vulnerable to NAFTA suffered large and persistent employment losses after its implementation. Voters in these counties (and protectionist voters regardless of geography) turned away from the party of President Clinton, who promoted the agreement. This shift is larger for whites (especially men and those without a college degree) and social conservatives, suggesting that racial identity and social-issue positions mediate reactions to economic policies.'),
 ('Nobel Lecture: An Evolving Economic Force',
  'Claudia Goldin',
  "South Koreans appear to be preoccupied with their offspring's education and also have the lowest total fertility rate in the world. We propose a novel theory with status externalities and endogenous fer

In [16]:
with open('aer/AER_' + issue + '.html','w',encoding='UTF-8') as f:
    f.write(gen_css.gen_content_from_list(titles))
    f.write(gen_css.gen_abstract_list(list(article)))
add_current.add('AER_' + issue, time.ctime(), 'aer/AER_' + issue + '.html')